In [1]:
import importlib
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [2]:
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import RDFlibConnection
from cimgraph.models import FeederModel
from cimgraph.models import NodeBreakerModel
from cimgraph.models import BusBranchModel
from cimgraph.tests.test_get_all_edges import test_get_all_edges

In [8]:
import cimgraph.utils as utils

In [61]:
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="../cimgraph/tests/test_models/ieee13.xml", cim_profile='cimhub_2023', iec61970_301=8)
rdf = RDFlibConnection(params)

feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=rdf, container=feeder, distributed=False)



In [62]:

utils.get_all_data(network)

ConnectivityNode
Terminal
EnergyConsumer
Breaker
ACLineSegment
PowerTransformer
Recloser
PowerElectronicsConnection
LoadBreakSwitch
LinearShuntCompensator
EnergySource
Fuse
Feeder
TopologicalNode
OperationalLimitSet
TransformerTankEnd
TapChangerControl
PowerTransformerEnd
TopologicalIsland
ACLineSegmentPhase
BaseVoltage
PerLengthPhaseImpedance
Location
PhaseImpedanceData
TransformerTank
TransformerTankInfo
RatioTapChanger
TransformerEndInfo
ShortCircuitTest
NoLoadTest
TransformerMeshImpedance
TransformerCoreAdmittance
EnergyConsumerPhase
LoadResponseCharacteristic
PowerElectronicsConnectionPhase
BatteryUnit
PhotovoltaicUnit
CurrentLimit
VoltageLimit
OperationalLimitType
PositionPoint
CoordinateSystem
SwitchPhase
LinearShuntCompensatorPhase
Substation
SubGeographicalRegion
GeographicalRegion


In [63]:
network.pprint(cim.BaseVoltage)

{
    "2A158E0C-CD01-4A50-AEBA-59D761FCF15D": {
        "mRID": "2A158E0C-CD01-4A50-AEBA-59D761FCF15D",
        "name": "BaseV_4.1600",
        "nominalVoltage": "4160",
        "ConductingEquipment": "['8CA46F28-84C0-4FE2-B228-EC4080777865', '47E52220-4596-4AF0-9724-0167B72A4DB8', '60B55036-DD71-40CE-ADDA-97B8CE7447DC', '52DE9189-20DC-4C73-BDEE-E960FE1F9493', '8E180773-2A9B-4136-BC9A-132A52C8E276', '3FFFB3F7-A7AF-49E1-ACEE-449FC73D3CD6', 'ABF877D7-DAC2-4BF0-AB58-9A8A02E92EB3', 'D34B0D01-B082-4081-A3CC-B68B9B8313A4', 'A04CDFB1-E951-4FC4-8882-0323CD70AE3C', 'ABF53597-A808-422A-B7EE-552F24D83A5F', 'FBE26B35-13AB-457D-9795-DF58B28E309D', '45395C84-F20A-4F5A-977F-B80348256421', 'CE5D0651-676B-4AF3-8D67-41BF1B33E30C', '517413CB-6977-46FA-8911-C82332E42884', 'A9DE8829-58CB-4750-B2A2-672846A89753', '9CAAF741-BE1E-4893-8044-1E507B7DDC38', '4C04F838-62AA-475E-AEFA-A63B7C889C13', '7060D0BB-B30D-4932-8FA1-40820A0FC4D0', '1C6781A2-5B9D-4525-8A9B-F9B32C4C4AC0', '9D725810-BFD6-44C6-961A-2BC027F6FC95

In [64]:
metrics = {}

if cim.Substation in network.graph:
    for substation in network.graph[cim.Substation].values():
        metrics['substation_name'] = substation.name
else:
    metrics['substation_name'] = None

metrics["lv_len_km"] = 0
metrics["mv_len_km"] = 0
metrics["mv_3ph_len_km"] = 0
metrics["mv_2ph_len_km"] = 0
metrics["mv_1ph_len_km"] = 0

for base_voltage in network.graph[cim.BaseVoltage].values():
    if float(base_voltage.nominalVoltage) > 1000:
    
        for equipment in base_voltage.ConductingEquipment:
            if equipment.__class__.__name__ == 'ACLineSegment':
                metrics["mv_len_km"] = metrics["mv_len_km"] + float(equipment.length)/1000
                phases = []
                for phase in equipment.ACLineSegmentPhases:
                    phases.append(str(phase.phase).split('SinglePhaseKind.')[1])
                    if 'N' in phases:
                        num_phases = len(phases) - 1
                    else:
                        num_phases = len(phases)
                    if 's1' in phases or 's2' in phases:
                        _log.warning(f'Triplex line over 1000 V with mRID {equipment.mRID}')
            

                if num_phases == 3:
                    metrics["mv_3ph_len_km"] = metrics["mv_3ph_len_km"] + float(equipment.length)/1000
                elif num_phases == 2:
                    metrics["mv_2ph_len_km"] = metrics["mv_2ph_len_km"] + float(equipment.length)/1000
                elif num_phases == 1:
                    metrics["mv_1ph_len_km"] = metrics["mv_1ph_len_km"] + float(equipment.length)/1000

metrics


{'substation_name': 'IEEE13',
 'lv_len_km': 0,
 'mv_len_km': 2.4993600000000002,
 'mv_3ph_len_km': 1.8288,
 'mv_2ph_len_km': 0.33528,
 'mv_1ph_len_km': 0.33528}

In [65]:
metrics['num_capacitors'] = len(network.graph[cim.LinearShuntCompensator].keys())
print(metrics['num_capacitors'])

2


In [66]:
metrics['num_regulators'] = len(network.graph[cim.RatioTapChanger].keys())

metrics['num_regulator_banks'] = 0
metrics['num_distribution_transformers'] = 0
metrics['num_3ph_transformers'] = 0
metrics['num_1ph_transformers'] = 0

for transformer in network.graph[cim.PowerTransformer].values():
    # for end in transformer.PowerTransformerEnd # skipping LTC's
    tap = False
    for tank in transformer.TransformerTanks:
        for end in tank.TransformerTankEnds:
            if end.RatioTapChanger is not None:
                tap = True
    if tap == True:
        metrics['num_regulator_banks'] += 1
    else: 
        metrics['num_distribution_transformers'] += 1

    if transformer.TransformerTanks == []:
        metrics['num_3ph_transformers'] += 1
    elif len(transformer.TransformerTanks) == 1:
        metrics['num_1ph_transformers'] += 1 #TODO: use terminals / windings count

    

metrics


{'substation_name': 'IEEE13',
 'lv_len_km': 0,
 'mv_len_km': 2.4993600000000002,
 'mv_3ph_len_km': 1.8288,
 'mv_2ph_len_km': 0.33528,
 'mv_1ph_len_km': 0.33528,
 'num_capacitors': 2,
 'num_regulators': 3,
 'num_regulator_banks': 1,
 'num_distribution_transformers': 3,
 'num_3ph_transformers': 2,
 'num_1ph_transformers': 1}

In [44]:
def count_device(network, class_type):
    if class_type in network.graph:
        count = len(network.graph[class_type].keys())
    else:
        count = 0
    return count

In [67]:
metrics['num_fuses'] = count_device(network, cim.Fuse)
metrics['num_sectionalizers'] = count_device(network, cim.Sectionaliser)
metrics['num_reclosers'] = count_device(network, cim.Recloser)
metrics['num_breakers'] = count_device(network, cim.Breaker)
metrics['num_switches'] = count_device(network, cim.LoadBreakSwitch) + count_device(network, cim.Switch)


metrics


{'substation_name': 'IEEE13',
 'lv_len_km': 0,
 'mv_len_km': 2.4993600000000002,
 'mv_3ph_len_km': 1.8288,
 'mv_2ph_len_km': 0.33528,
 'mv_1ph_len_km': 0.33528,
 'num_capacitors': 2,
 'num_regulators': 3,
 'num_regulator_banks': 1,
 'num_distribution_transformers': 3,
 'num_3ph_transformers': 2,
 'num_1ph_transformers': 1,
 'num_fuses': 1,
 'num_sectionalizers': 0,
 'num_reclosers': 1,
 'num_breakers': 1,
 'num_switches': 2}

In [76]:
metrics['sum_load_kw'] = 0
metrics['sum_load_kvar'] = 0

for load in network.graph[cim.EnergyConsumer].values():
    metrics['sum_load_kw'] = metrics['sum_load_kw'] + float(load.p)
    metrics['sum_load_kvar'] = metrics['sum_load_kvar'] + float(load.q)
    
 
    


In [78]:
dist_network = FeederModel(connection=rdf, container=feeder, distributed=True)

In [82]:
for switch_area in dist_network.distributed_areas:
    for secondary_area in switch_area.distributed_areas:
        utils.get_all_data(secondary_area)
        
    switch_area.pprint(cim.PowerTransformer)


{
    "67B57539-590B-4158-9CBB-9DBA2FE6C1F0": {
        "mRID": "67B57539-590B-4158-9CBB-9DBA2FE6C1F0",
        "name": "Reg",
        "Location": "16F0DA5A-A603-45D1-B14B-DA3EFF209FD3",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "Terminals": "['02CD5661-C654-43F6-8D1A-0D666070BB81', '5A1120EE-0F2A-4698-9FF5-0F0F4F507BAF', '9AC596A8-D859-48F3-A286-D2533F9B9733', 'F8856652-20FB-4760-874D-3D785E91A83E', 'EC150DA0-864E-4C61-B681-3CF27AB27B82', 'F5258B9A-B51E-40DD-9A06-5918E41F3C35']",
        "vectorGroup": "Yy",
        "TransformerTanks": "['44FC5A86-A099-45B8-B847-F685C5027AFB', 'E2E0FC64-8D45-4C55-BDB9-EAB827A46FBC', 'B6363F07-B1BC-420B-AA4C-A34BB8F05827']"
    }
}
{
    "259E820F-B4AF-4E1A-8271-687534EDAECC": {
        "mRID": "259E820F-B4AF-4E1A-8271-687534EDAECC",
        "name": "sub3",
        "Location": "6CE06D43-8DE4-4C72-B059-F71EAE02DDDD",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "Terminals": "['F6CE3095

In [80]:
switch_area.

NameError: name 'secondary_area' is not defined

In [83]:
metrics

{'substation_name': 'IEEE13',
 'lv_len_km': 0,
 'mv_len_km': 2.4993600000000002,
 'mv_3ph_len_km': 1.8288,
 'mv_2ph_len_km': 0.33528,
 'mv_1ph_len_km': 0.33528,
 'num_capacitors': 2,
 'num_regulators': 3,
 'num_regulator_banks': 1,
 'num_distribution_transformers': 3,
 'num_3ph_transformers': 2,
 'num_1ph_transformers': 1,
 'num_fuses': 1,
 'num_sectionalizers': 0,
 'num_reclosers': 1,
 'num_breakers': 1,
 'num_switches': 2,
 'sum_load_kw': 3471000.0,
 'sum_load_kvar': 2105098.7217}

In [72]:
network.pprint(cim.EnergyConsumer)

{
    "60B55036-DD71-40CE-ADDA-97B8CE7447DC": {
        "mRID": "60B55036-DD71-40CE-ADDA-97B8CE7447DC",
        "name": "611",
        "Location": "08866A9B-A318-4DA1-83FE-16EAD7927BA9",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "BaseVoltage": "2A158E0C-CD01-4A50-AEBA-59D761FCF15D",
        "Terminals": "['2F5C9D9B-8CC5-4636-8BC8-2AC78F01F5BC']",
        "customerCount": "1",
        "grounded": "true",
        "p": "170000",
        "phaseConnection": "PhaseShuntConnectionKind.Y",
        "q": "80000",
        "EnergyConsumerPhase": "['F7A295B8-655E-4C2C-95BA-0F52F81EB404']",
        "LoadResponse": "7E09238A-DF87-4F3A-A192-4FA4C4416A4F"
    },
    "3FFFB3F7-A7AF-49E1-ACEE-449FC73D3CD6": {
        "mRID": "3FFFB3F7-A7AF-49E1-ACEE-449FC73D3CD6",
        "name": "675c",
        "Location": "5F4F5E06-9447-448B-8DE4-F4E22AA34576",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "BaseVoltage": "2A158E0C-CD01-4A50-AEBA-59D761

In [53]:
network.pprint(cim.PowerTransformer)

{
    "259E820F-B4AF-4E1A-8271-687534EDAECC": {
        "mRID": "259E820F-B4AF-4E1A-8271-687534EDAECC",
        "name": "sub3",
        "Location": "6CE06D43-8DE4-4C72-B059-F71EAE02DDDD",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "Terminals": "['F6CE3095-9089-4B61-9C13-C7647ADBA888', '717DFB1A-A300-444D-8D3B-0A093EAAD47B', 'BB0411D7-5261-433D-B327-549DA536EEEC']",
        "vectorGroup": "Dyn1y1",
        "PowerTransformerEnd": "['19B21AD3-7795-4477-8232-74DAEC0DF7C9', 'D103813B-F494-47B7-8A91-BA4EED4D61E4', '7F72BD91-DA7F-42D7-96F8-2E2FEBA9E4AB']"
    },
    "67B57539-590B-4158-9CBB-9DBA2FE6C1F0": {
        "mRID": "67B57539-590B-4158-9CBB-9DBA2FE6C1F0",
        "name": "Reg",
        "Location": "16F0DA5A-A603-45D1-B14B-DA3EFF209FD3",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "Terminals": "['02CD5661-C654-43F6-8D1A-0D666070BB81', '5A1120EE-0F2A-4698-9FF5-0F0F4F507BAF', '9AC596A8-D859-48F3-A286-D2533F9B9733', 'F885

In [35]:
network.pprint(cim.TransformerTankEnd)

{
    "E5033D65-F871-4710-A0BE-0D62D8FB6DE4": {
        "mRID": "E5033D65-F871-4710-A0BE-0D62D8FB6DE4",
        "name": "reg2_End_2",
        "endNumber": "2",
        "grounded": "true",
        "rground": "0",
        "xground": "0",
        "BaseVoltage": "2A158E0C-CD01-4A50-AEBA-59D761FCF15D",
        "RatioTapChanger": "C03B9EC2-DD5F-4C39-A4A7-E212CBBC9940",
        "Terminal": "02CD5661-C654-43F6-8D1A-0D666070BB81",
        "orderedPhases": "OrderedPhaseCodeKind.BN",
        "TransformerTank": "44FC5A86-A099-45B8-B847-F685C5027AFB"
    },
    "3647258E-0387-427F-98D9-162BC772B4EB": {
        "mRID": "3647258E-0387-427F-98D9-162BC772B4EB",
        "name": "reg1_End_2",
        "endNumber": "2",
        "grounded": "true",
        "rground": "0",
        "xground": "0",
        "BaseVoltage": "2A158E0C-CD01-4A50-AEBA-59D761FCF15D",
        "RatioTapChanger": "6CFE3829-3905-42FD-A9AD-5C70D3D835EF",
        "Terminal": "5A1120EE-0F2A-4698-9FF5-0F0F4F507BAF",
        "orderedPhases": 

In [17]:
# Get everything related to buses
network.get_all_edges(cim.Terminal)
network.get_all_edges(cim.ConnectivityNode)
network.get_all_edges(cim.Location)
network.get_all_edges(cim.PositionPoint)
network.get_all_edges(cim.Feeder)
network.get_all_edges(cim.Substation)

In [11]:
import logging
_log = logging.getLogger(__name__)

In [15]:
# Get capacitor data
network.get_all_edges(cim.LinearShuntCompensator)
network.get_all_edges(cim.LinearShuntCompensatorPhase)
network.get_all_edges(cim.RegulatingControl)

41.6666664

In [22]:
# get ditto capacitor info

for element in network.graph[cim.LinearShuntCompensator].values():
    susceptance = float(element.bPerSection)
    conductance = float(element.gPerSection)
    # resistance = 1 / conductance #not in CIM
    susceptance0 = float(element.b0PerSection)
    conductance0 = float(element.g0PerSection)
    # resistance0 = 1 / conductance0
    connecting_element = element.Terminals[0].ConnectivityNode

    if element.RegulatingControl is not None:
        high = element.RegulatingControl #TODO should  be vmax in opendss -- figure out Tom's converter 
        low = element.RegulatingControl #TODO should  be vmax in opendss -- figure out Tom's converter 
        mode = element.RegulatingControl.mode #enum
    delay = float(element.aVRDelay)
    nominal_voltage = float(element.nomU)

    positions = element.Location.PositionPoints

    measuring_element = None #TODO associate with OpenDSS line object meas
    
    pt_ratio = None #missing?
    pt_phase = None
    ct_ratio = None #missing?
    

    connection_type = element.phaseConnection

    container = element.EquipmentContainer
    if container.__class__.__name__ == "Feeder":
        feeder_name = container.name
        substation_name = container.NormalEnergizingSubstation.name
        is_substation = False
    elif container.__class__.__name__ == "Substation":
        feeder_name = None
        substation_name = container.name
        is_substation = True
    else:
        feeder_name = None
        try:
            substation_name = container.Substation.name
        except:
            _log.warning("could not identify feeder or substation")

    for phase in element.ShuntCompensatorPhase:
        switch = phase.sections #TODO: figure out difference between switch and sections
        sections = phase.sections 
        normalsections = phase.normalSections
        var = nominal_voltage * nominal_voltage * float(phase.bPerSection)

        



In [36]:
nominal_voltage * nominal_voltage * float(phase.bPerSection)


99999.99935999999

In [9]:
network.pprint(cim.LinearShuntCompensator, show_empty=True)

{
    "A9DE8829-58CB-4750-B2A2-672846A89753": {
        "mRID": "A9DE8829-58CB-4750-B2A2-672846A89753",
        "aliasName": "",
        "description": "",
        "name": "cap1",
        "Names": "",
        "AssetDatasheet": "",
        "Assets": "",
        "ConfigurationEvent": "",
        "Controls": "",
        "Location": "62A249A4-4BF9-42BB-9E0A-06D132216D9A",
        "Measurements": "",
        "OperatingShare": "",
        "PSRType": "",
        "ReportingGroup": "",
        "aggregate": "",
        "inService": "",
        "networkAnalysisEnabled": "",
        "normallyInService": "",
        "AdditionalEquipmentContainer": "",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "Faults": "",
        "OperationalLimitSet": "",
        "UsagePoints": "",
        "BaseVoltage": "2A158E0C-CD01-4A50-AEBA-59D761FCF15D",
        "SvStatus": "",
        "Terminals": "['5B38070B-E918-4EAA-8BDB-8B3CE4F8A917']",
        "EnergyConnectionProfile": "",
        

In [5]:
# Get everything related to transformers
network.get_all_edges(cim.PowerTransformer)
network.get_all_edges(cim.TransformerTank)
network.get_all_edges(cim.TransformerTankEnd)
network.get_all_edges(cim.TransformerTankInfo)
network.get_all_edges(cim.TransformerEndInfo)
network.get_all_edges(cim.PowerTransformerEnd)
network.get_all_edges(cim.PowerTransformerInfo)
network.get_all_edges(cim.TransformerCoreAdmittance)
network.get_all_edges(cim.TransformerMeshImpedance)
network.get_all_edges(cim.TransformerStarImpedance)
network.get_all_edges(cim.ShortCircuitTest)
network.get_all_edges(cim.NoLoadTest)
network.get_all_edges(cim.RatioTapChanger)
network.get_all_edges(cim.TapChanger)
network.get_all_edges(cim.TapChangerControl)
network.get_all_edges(cim.TapChangerInfo)

In [ ]:
# Power Transformer stuff
#element
for element in network.graph[cim.PowerTransformer]:
    for winding in element.PowerTransformerEnds:
        to_element = winding.Terminals[0].ConnectivityNode #obj
        to_element_connection_index = winding.endNumber #float, 1 is high-side
        from_element = winding.Terminals[1].ConnectivityNode #obj
    
    feeder_name = element.EquipmentContainer.name #str



In [4]:
import cimgraph.utils as utils

In [5]:
utils.get_all_data(network)

ConnectivityNode
Terminal
EnergyConsumer
Breaker
ACLineSegment
PowerTransformer
Recloser
PowerElectronicsConnection
LoadBreakSwitch
LinearShuntCompensator
EnergySource
Fuse
Feeder
TopologicalNode
OperationalLimitSet
TransformerTankEnd
TapChangerControl
PowerTransformerEnd
TopologicalIsland
ACLineSegmentPhase
BaseVoltage
PerLengthPhaseImpedance
Location
PhaseImpedanceData
TransformerTank
TransformerTankInfo
RatioTapChanger
TransformerEndInfo
ShortCircuitTest
NoLoadTest
TransformerMeshImpedance
TransformerCoreAdmittance
EnergyConsumerPhase
LoadResponseCharacteristic
PowerElectronicsConnectionPhase
BatteryUnit
PhotovoltaicUnit
CurrentLimit
VoltageLimit
OperationalLimitType
PositionPoint
CoordinateSystem
SwitchPhase
LinearShuntCompensatorPhase
Substation
SubGeographicalRegion
GeographicalRegion
